In [1]:
from astropy.io import fits
from astropy.table import Table, vstack

import numpy as np
import os
import uuid

In [ ]:
root_path = "/home/mike/git/computational_astro/astro_iqa"
data_path = "data/raw/ngc7000"
data_path = os.path.join(root_path, data_path)

# List all fits files in the data directory
fits_files = [f for f in os.listdir(data_path) if f.endswith(".fit")]

In [16]:
# For each fits file, generate ldac file with sextractor
# config_file = "xymfhe.sex"
# for fits_file in fits_files:
#     fits_path = os.path.join(data_path, fits_file)
#     ldac_path = fits_path.replace(".fit", ".ldac")
#     if not os.path.exists(ldac_path):
#         os.system("sex {} -c {}".format(fits_path, config_file))
#         os.system("mv {} {}".format("test.cat", ldac_path))

In [17]:
def log_scale_data (data, mini, maxi):
    data = np.where(data < mini, mini, data)
    data = np.where(data > maxi, maxi, data)
    data = np.log10(data)
    return data

def remove_outliers (data, mini, maxi):
    data = np.where(data < mini, mini, data)
    data = np.where(data > maxi, maxi, data)
    return data

In [ ]:
# objects_catalog = "som_objects_catalog_ngc7000.hdf5"
objects_catalog = "objects_catalog_ngc7000_bronze.parquet.gz"
# catalog_path = os.path.join(root_path, "data/processed")
catalog_path = os.path.join(root_path, "data/for_modeling")
filename = os.path.join(catalog_path, objects_catalog)

# Initializing the global catalog
catalog = Table(names=("FITS_ID", "CCD_ID", "OBJECT_ID", "ISO0", "BACKGROUND", "ELLIPTICITY", "ELONGATION", "CLASS_STAR", "FLAGS", "EXPTIME"), 
    dtype=("S12", np.uint8, "S32", np.float32, np.float32, np.float32, np.float32, np.float32, np.int16, np.float32))
# For each ldac file, read each table of each ccd
for ldac_file in [f for f in os.listdir(data_path) if f.endswith(".ldac")]:
    fits_id, extension = os.path.splitext(ldac_file)
    ldac_path = os.path.join(data_path, ldac_file)
    print("Processing {}".format(ldac_path))
    ldac = fits.open(ldac_path)
    # print(ldac.info())
    ldac_tables = [hdu for hdu in ldac if isinstance(hdu, fits.BinTableHDU)]
    n_ccd = len(ldac_tables)
    fits_ = fits.open(os.path.join(data_path, fits_id+".fit"))
    for i in range(1, len(ldac)):
        if ldac[i].data is not None and ldac[i].data.shape[0] > 0:
            table = Table(ldac[i].data)
            table.add_column(fits_id, name="FITS_ID", index=0)
            table.add_column(np.uint8(i), name="CCD_ID", index=1)
            ids = [str(uuid.uuid4().hex) for i in range(ldac[i].data.shape[0])]
            table.add_column(ids, name="OBJECT_ID", index=2) # there is a bug in the generation of the unique id, it should be unique for each row of the table
            # TODO: fix the bug in the generation of the unique id by using the equialent of the function apply in pandas            
            try:
                exptime = fits_[i].header["EXPTIME"]
            except:
                exptime = 30
            table.add_column(exptime, name="EXPTIME")
            # normalise data before their registration
            try:
                table["ISO0"] = table["ISO0"].astype(np.float32)
                # table["ISO0"] = log_scale_data(table["ISO0"], 0.00001, 10000)
                table["ELLIPTICITY"] = remove_outliers(table["ELLIPTICITY"], 0.00001, 1)
                # table["EXPTIME"] = log_scale_data(exptime * np.abs(table["BACKGROUND"]/np.mean(table["BACKGROUND"])), 0.00001, 30)
                table["BACKGROUND"] = remove_outliers((table["BACKGROUND"]-np.mean(table["BACKGROUND"])/np.std(table["BACKGROUND"])), -2., 2.)
                # print("Updating catalog file in {}".format(data_path))
                # integration table data into the catalog
                # print(table.info())
                catalog = vstack([catalog, table])
            except:
                # In case one the ldac tables is empty or corrupted, we skip it
                continue
    # print(catalog.info())
    fits_.close()
    ldac.close()



Processing /home/mike/git/computational_astro/astro_iqa/data/raw/ngc0896/ngc0896_220524_raw_00001.ldac
Processing /home/mike/git/computational_astro/astro_iqa/data/raw/ngc0896/ngc0896_220524_raw_00002.ldac
Processing /home/mike/git/computational_astro/astro_iqa/data/raw/ngc0896/ngc0896_220524_raw_00003.ldac
Processing /home/mike/git/computational_astro/astro_iqa/data/raw/ngc0896/ngc0896_220524_raw_00004.ldac
Processing /home/mike/git/computational_astro/astro_iqa/data/raw/ngc0896/ngc0896_220524_raw_00005.ldac
Processing /home/mike/git/computational_astro/astro_iqa/data/raw/ngc0896/ngc0896_220524_raw_00006.ldac


In [19]:
# Save the catalog
# catalog.write(filename, path="som_catalog", format="hdf5", overwrite=True, append=True)
catalog_df = catalog.to_pandas()
catalog_df.to_parquet(filename, compression="gzip", engine="auto")